# Top

In [1]:
!pip install transformers
from google.colab import drive
drive.mount('/content/drive')

     |████████████████████████████████| 2.3MB 7.5MB/s 
     |████████████████████████████████| 3.3MB 50.1MB/s 
     |████████████████████████████████| 901kB 50.5MB/s 
Mounted at /content/drive


In [2]:
import os
import pandas as pd

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset

from transformers import RobertaModel, RobertaTokenizer, AdamW

from tqdm.auto import tqdm

# For visualizations
import matplotlib.pyplot as plt
import seaborn as sns

cur_dir = "/content/drive/MyDrive/Colab_Notebooks/NLP_244_Advanced_ML/final_project_socialiqa/socialiqa-nlp244"
data_dir = "socialiqa-train-dev"
out_dir = "out"

# Extract SocialIQA data

In [3]:
file_train = os.path.join(cur_dir, "socialiqa-train-dev/train.jsonl")
file_dev = os.path.join(cur_dir, "socialiqa-train-dev/dev.jsonl")

json_train = pd.read_json(path_or_buf=file_train, lines=True)
json_dev = pd.read_json(path_or_buf=file_dev, lines=True)

# list of tuples (context, question, A, B++++, C)
train_data = [elem for elem in zip(json_train['context'].tolist(), 
                                   json_train['question'].tolist(), 
                                   json_train['answerA'].tolist(), 
                                   json_train['answerB'].tolist(), 
                                   json_train['answerC'].tolist())]

dev_data = [elem for elem in zip(json_dev['context'].tolist(), 
                                   json_dev['question'].tolist(), 
                                   json_dev['answerA'].tolist(), 
                                   json_dev['answerB'].tolist(), 
                                   json_dev['answerC'].tolist())]

len(train_data), len(dev_data)

(33410, 1954)

In [4]:
train_labels = []
dev_labels = []
with open(os.path.join(cur_dir, data_dir, "train-labels.lst")) as f:
    for line in f:
      train_labels.append(int(line.split()[0]))

with open(os.path.join(cur_dir, data_dir, "dev-labels.lst")) as f:
    for line in f:
      dev_labels.append(int(line.split()[0]))

train_labels = [label-1 for label in train_labels]
dev_labels = [label-1 for label in dev_labels]

len(train_labels), len(dev_labels)

(33410, 1954)

In [5]:
class SocialiqaDataset(Dataset):
    """
    This dataset class for socialiqa might be able to be generalized for 
    HellaSwag and other tasks.

    This is the context/question + multiple choice format, and each example
    consists of num choices lists of encoded strings. Note that the input will
    be encoded in this stage. prepare_batch will take care of padding across examples
    in the batch-level. 
    """
    def __init__(self, tokenizer, x, y):
        # x: list of tuples containing (context, question, answer1, answer2, answer3)
        # y: list of indices of the correct answer
        self.roberta_tokenizer = tokenizer
        self.x = x
        self.y = y

    def __getitem__(self, idx):
        point = self.x[idx]
        input_context_question = [point[0] + self.roberta_tokenizer.sep_token + self.roberta_tokenizer.sep_token + point[1], point[0] + self.roberta_tokenizer.sep_token + self.roberta_tokenizer.sep_token + point[1], point[0] + self.roberta_tokenizer.sep_token + self.roberta_tokenizer.sep_token + point[1]]
        input_answers = [point[2], point[3], point[4]]
        encoded_text_train = self.roberta_tokenizer(input_context_question, input_answers, return_tensors='pt', padding=True)
        return (encoded_text_train, self.y[idx])

    def __len__(self):
        return len(self.x)


def prepare_batch_MC(batch, tokenizer):
    """
    This collate function will pad the batch to be the same length. This requires
    flattening, then unflattening for the multiple choice format.
    One example will be a list of length 'num choices', each element being a list
    of (encoded) tokens representing qustion/answer [sep] choicex
    """
    # batch: [batch_size, (text, label)]
    batch_size = len(batch)

    features, labels = zip(*batch)
    # features: tuple of length batch_size, 
    #        each element is a dict with keys = ["input_ids", "attention_mask"]
    # labels: tuple of ints (0, 1, 2) of length batch_size
    num_choices = len(features[0]["input_ids"])
    
    # flatten
    flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
    flattened_features = sum(flattened_features, [])
    # flattened_features list length num_choices*batch_size

    batch = tokenizer.pad(
            flattened_features,
            padding=True,
            return_tensors="pt",
        )
    
    # Un-flatten
    batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
    return (batch, torch.tensor(labels, dtype=torch.int64))


In [6]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
dataset_train = SocialiqaDataset(tokenizer, train_data, train_labels)
dataset_dev = SocialiqaDataset(tokenizer, dev_data, dev_labels)

# Model Class

In [7]:
class Multiple_Choice_Model(nn.Module):
    def __init__(self, roberta_model: RobertaModel, dropout: float = None):
          super(Multiple_Choice_Model, self).__init__()
          self.roberta = roberta_model
          self.dropout = nn.Dropout(self.roberta.config.hidden_dropout_prob)
          self.classifier = nn.Linear(self.roberta.config.hidden_size, 1)

    def forward(self, input_ids: torch.tensor, attention_mask: torch.tensor, labels=None):
          num_choices = input_ids.shape[1] 
          flat_input_ids = input_ids.view(-1, input_ids.size(-1))
          flat_attention_mask = attention_mask.view(-1, attention_mask.size(-1))

          outputs = self.roberta(
              flat_input_ids,
              attention_mask=flat_attention_mask,
          )
          pooled_output = outputs[1] 

          pooled_output = self.dropout(pooled_output)
          logits = self.classifier(pooled_output)
          reshaped_logits = logits.view(-1, num_choices)

          loss = None
          loss_fct = nn.CrossEntropyLoss()
          loss = loss_fct(reshaped_logits, labels)

          return loss, reshaped_logits

In [31]:
class Trainer(object):
    """
    Trainer for training a multiple choice classification model
    """

    def __init__(self, model, optimizer, device="cpu"):
        self.model = model.to(device)
        self.optimizer = optimizer
        self.device = device

    def _print_summary(self):
        print(self.model)
        print(self.optimizer)

    def train(self, loader):
        """
        Run a single epoch of training
        """

        self.model.train() # Run model in training mode
        loss = None
        for i, batch in tqdm(enumerate(loader)):
            # clear gradient
            self.optimizer.zero_grad() 

            # input_ids shape: (batch_size, num_choices, sequence_length)
            input_ids = batch[0]['input_ids'].to(self.device)
            # input_ids shape: (batch_size, num_choices, sequence_length)
            attention_mask = batch[0]['attention_mask'].to(self.device)
            # labels shape: (batch_size, )
            labels = batch[1].to(self.device)

            outputs = self.model(input_ids=input_ids, 
                            attention_mask=attention_mask,
                            labels=labels)
            loss, logits = outputs[0], outputs[1]
            
            # back propagation
            loss.backward()
            # do gradient descent
            self.optimizer.step() 

        # Just returning the last loss
        return loss

    def evaluate(self, loader):
        """
        Evaluate the model on a validation set.
        Only do batch size = 1.
        """

        self.model.eval() # Run model in eval mode (disables dropout layer)
        loss = None
        with torch.no_grad(): # Disable gradient computation - required only during training
            for i, batch in tqdm(enumerate(loader)):
                # input_ids shape: (batch_size, num_choices, sequence_length)
                input_ids = batch[0]['input_ids'].to(self.device)
                # input_ids shape: (batch_size, num_choices, sequence_length)
                attention_mask = batch[0]['attention_mask'].to(self.device)
                # labels shape: (batch_size, )
                labels = batch[1].to(self.device)

                outputs = self.model(input_ids=input_ids, 
                                attention_mask=attention_mask,
                                labels=labels)
                loss, logits = outputs[0], outputs[1]

        # Just returning the last loss
        return loss

    def get_model_dict(self):
        return self.model.state_dict()

    def run_training(self, train_loader, valid_loader, n_epochs=3):
        # Useful for us to review what experiment we're running
        # Normally, you'd want to save this to a file
        # self._print_summary()
        losses_valid = []
        losses_train = []
        best_valid = float("inf")
        for i in range(n_epochs):
            epoch_loss_train = self.train(train_loader)
            epoch_loss_valid = self.evaluate(valid_loader)
            if epoch_loss_valid < best_valid:
                best_valid = epoch_loss_valid
                torch.save(self.get_model_dict(), os.path.join(cur_dir, out_dir, f'model-mc-checkpoint-epoch{i+1}.pt'))
            losses_train.append(epoch_loss_train.tolist())
            losses_valid.append(epoch_loss_valid.tolist())
            print(f"Epoch {i}")
            print(f"Train loss: {epoch_loss_train}")
            print(f"Valid loss: {epoch_loss_valid}")

        train_epoch_idx = range(len(losses_train))
        valid_epoch_idx = range(len(losses_valid))
        # sns.lineplot(epoch_idx, all_losses)
        sns.lineplot(train_epoch_idx, losses_train)
        sns.lineplot(valid_epoch_idx, losses_valid)
        plt.show()
        

In [29]:
roberta_base = RobertaModel.from_pretrained('roberta-base')
mc_model = Multiple_Choice_Model(roberta_base)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# 'W' stands for 'Weight Decay fix"
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
optimizer = AdamW(mc_model.parameters(), lr=5e-5)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [30]:
train_loader = DataLoader(dataset_train, batch_size=16, shuffle=True, collate_fn=lambda batch: prepare_batch_MC(batch, tokenizer))
val_loader = DataLoader(dataset_dev, batch_size=1, shuffle=False)

# Training

In [ ]:
trainer_socialiqa = Trainer(mc_model, optimizer, device)
trainer_socialiqa.run_training(train_loader, val_loader, n_epochs=10)


Epoch 0
Train loss: 1.0864859819412231
Valid loss: 1.0986131429672241



Epoch 1
Train loss: 1.0808427333831787
Valid loss: 1.098609447479248



Epoch 2
Train loss: 1.1127394437789917
Valid loss: 1.0986131429672241



Epoch 3
Train loss: 1.1298034191131592
Valid loss: 1.0986119508743286


In [10]:
mc_model_restored_state_dict = torch.load(os.path.join(cur_dir, out_dir, 'model-mc-checkpoint-epoch3'), )

roberta_base = RobertaModel.from_pretrained('roberta-base')
mc_model_restored = Multiple_Choice_Model(roberta_base)

# Restore state dict to load the same weights again
mc_model_restored.load_state_dict(mc_model_restored_state_dict)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [27]:
mc_model_restored.to(device)
mc_model_restored.eval()
val_loader = DataLoader(dataset_dev, batch_size=1, shuffle=False)
for i, batch in tqdm(enumerate(val_loader)):
                # input_ids shape: (batch_size, num_choices, sequence_length)
                input_ids = batch[0]['input_ids'].to(device)
                # input_ids shape: (batch_size, num_choices, sequence_length)
                attention_mask = batch[0]['attention_mask'].to(device)
                # labels shape: (batch_size, )
                labels = batch[1].to(device)

                outputs = mc_model_restored(input_ids=input_ids, 
                                attention_mask=attention_mask,
                                labels=labels)
                _, logits = outputs[0], outputs[1]
                print(logits)
                

tensor([[0.2371, 0.2371, 0.2371]], device='cuda:0', grad_fn=<ViewBackward>)
tensor([[0.2371, 0.2371, 0.2371]], device='cuda:0', grad_fn=<ViewBackward>)
tensor([[0.2371, 0.2371, 0.2371]], device='cuda:0', grad_fn=<ViewBackward>)
tensor([[0.2371, 0.2371, 0.2371]], device='cuda:0', grad_fn=<ViewBackward>)
tensor([[0.2371, 0.2371, 0.2371]], device='cuda:0', grad_fn=<ViewBackward>)
tensor([[0.2371, 0.2371, 0.2371]], device='cuda:0', grad_fn=<ViewBackward>)
tensor([[0.2371, 0.2371, 0.2371]], device='cuda:0', grad_fn=<ViewBackward>)
tensor([[0.2371, 0.2371, 0.2371]], device='cuda:0', grad_fn=<ViewBackward>)
tensor([[0.2371, 0.2371, 0.2371]], device='cuda:0', grad_fn=<ViewBackward>)
tensor([[0.2371, 0.2371, 0.2371]], device='cuda:0', grad_fn=<ViewBackward>)
tensor([[0.2371, 0.2371, 0.2371]], device='cuda:0', grad_fn=<ViewBackward>)
tensor([[0.2371, 0.2371, 0.2371]], device='cuda:0', grad_fn=<ViewBackward>)
tensor([[0.2371, 0.2371, 0.2371]], device='cuda:0', grad_fn=<ViewBackward>)
tensor([[0.2

KeyboardInterrupt: ignored